In [69]:

import numpy as np
import pandas as pd


from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.linear_model import SGDClassifier
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV

In [72]:
train_df = pd.read_csv(r'D:\Загрузки\train_sessions.csv\train_sessions.csv', index_col='session_id')

In [73]:
test_df = pd.read_csv(r'D:\Загрузки\test_sessions.csv\test_sessions.csv', index_col='session_id')

In [74]:
# В обучающей выборке train_sessions.csv:

# Признаки site_i – это индексы посещенных сайтов (расшифровка дана в pickle-файле со словарем site_dic.pkl)
# Признаки time_j – время посещения сайтов site_j
# Целевой признак target – факт того, что сессия принадлежит Элис (то есть что именно Элис ходила по всем этим сайтам)

In [75]:
train_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253557,3474,2013-11-25 10:26:54,3474.0,2013-11-25 10:26:58,141.0,2013-11-25 10:27:03,2428.0,2013-11-25 10:27:04,106.0,2013-11-25 10:27:13,...,2013-11-25 10:27:16,2428.0,2013-11-25 10:27:28,2428.0,2013-11-25 10:27:40,2428.0,2013-11-25 10:27:52,148.0,2013-11-25 10:27:53,0
253558,12727,2013-03-12 16:01:15,12727.0,2013-03-12 16:01:16,2215.0,2013-03-12 16:01:16,38.0,2013-03-12 16:01:17,2215.0,2013-03-12 16:01:17,...,2013-03-12 16:01:17,25444.0,2013-03-12 16:01:18,2215.0,2013-03-12 16:01:18,23.0,2013-03-12 16:01:18,21.0,2013-03-12 16:01:18,0
253559,2661,2013-09-12 14:05:03,15004.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:06:29,5562.0,2013-09-12 14:06:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [76]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [77]:
train_df.shape

(253561, 21)

In [78]:
test_df.shape

(82797, 20)

In [79]:
#Создание единого фрейма для удобства разработки фич, без повтора на тестовую выборку отдельно 

In [80]:
#отделить таргет от Трэйна и содеинить Трэйн с Тестом. Сумма строк составит 336 358

In [81]:
DF_Full = pd.concat([train_df.drop('target', axis=1), test_df])

In [82]:
DF_Full.shape

(336358, 20)

In [83]:
DF_Full.head(5)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11


In [84]:
#Фичаинжиниринг
time = ['time%s' % i for i in range(1,11)]
DF_Full = DF_Full.drop(time, axis=1)

In [98]:
DF_Full=DF_Full.fillna(0)
DF_Full = preprocessing.OneHotEncoder().fit_transform(DF_Full)

In [100]:
DF_Full

<336358x183199 sparse matrix of type '<class 'numpy.float64'>'
	with 3363580 stored elements in Compressed Sparse Row format>

In [68]:
DF_Full.site1.nunique()

18390

In [ ]:
#Отсоединение измененной Обучающей выборки от измененной целой и обучение моделей 

In [101]:
X = DF_Full[:253561]
X

<253561x183199 sparse matrix of type '<class 'numpy.float64'>'
	with 2535610 stored elements in Compressed Sparse Row format>

In [102]:
y = train_df.target

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=17)

In [ ]:
МОДЕЛИ:

In [109]:
Logistic1 = LogisticRegression(C=1.0, n_jobs=-1)

In [110]:
sgd1 = SGDClassifier(random_state=17, max_iter=500, class_weight='balanced', loss='log')

In [124]:
def Modelauc (model1, model2, X_train, X_test, y_train, y_test):
    pok = []
    model_list = [model1, model2]
    for model in model_list:
        model.fit(X_train, y_train)
        #ab = model.predict_proba(X_test)
        a = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]);
        b = precision_score(y_test, model.predict(X_test));
        c = recall_score(y_test, model.predict(X_test));

        pok.append(model.predict_proba(X_test)[:, 1]);

        print('For model', str(model)[0:11], 'ROC_AUC=', a)
        print('For model', str(model)[0:11], 'Prec=', b)
        print('For model', str(model)[0:11], 'Recall=', c)
        print('____________________________________________________')
        print('                                                    ')
    # return(pok)

    tablet = pd.DataFrame({'aa':pok[0], 'ab': pok[1]})
    tablet['Res'] = (tablet.aa + tablet.ab)/2
    print(tablet)
    print(roc_auc_score(y_test, tablet['Res']))

   

In [125]:
Modelauc(Logistic1, sgd1, X_train, X_test, y_train, y_test)

For model LogisticReg ROC_AUC= 0.9383363558557221
For model LogisticReg Prec= 0.7297297297297297
For model LogisticReg Recall= 0.06178489702517163
____________________________________________________
                                                    
For model SGDClassifi ROC_AUC= 0.9317483281758379
For model SGDClassifi Prec= 0.06935927257348276
For model SGDClassifi Recall= 0.7505720823798627
____________________________________________________
                                                    
             aa        ab       Res
0      0.001761  0.090664  0.046212
1      0.008673  0.218640  0.113656
2      0.000600  0.024489  0.012544
3      0.000326  0.013665  0.006995
4      0.000078  0.002968  0.001523
...         ...       ...       ...
50708  0.009322  0.263784  0.136553
50709  0.003351  0.152095  0.077723
50710  0.001568  0.054443  0.028006
50711  0.000393  0.018858  0.009626
50712  0.000066  0.002583  0.001324

[50713 rows x 3 columns]
0.9344063333329087


In [ ]:
#Отсоединение измененной тестовой выборки от измененной целой

In [106]:
Finial_Test = DF_Full[253561:]
Finial_Test

<82797x183199 sparse matrix of type '<class 'numpy.float64'>'
	with 827970 stored elements in Compressed Sparse Row format>

In [108]:
pd.DataFrame({'session_id': range(1, 82798), 'target': Logistic1.predict_proba(Finial_Test)[:, 1]}).set_index('session_id').to_csv('Try1_Catch_Alice.csv')